# 模型验证套路

In [1]:
import torchvision
from PIL import Image
from model import *
import torch

### torchvision.transforms.Resize()方法

```python
class torchvision.transforms.Resize(size, interpolation=InterpolationMode.BILINEAR, max_size=None, antialias=True)
Resize the input image to the given size. If the image is torch Tensor, it is expected to have […, H, W] shape, where … means a maximum of two leading dimensions

Parameters:

        size (sequence or int) –

        Desired output size. If size is a sequence like (h, w), output size will be matched to this. If size is an int, smaller edge of the image will be matched to this number. i.e, if height > width, then image will be rescaled to (size * height / width, size).

        Note

        In torchscript mode size as single int is not supported, use a sequence of length 1: [size, ].

        interpolation (InterpolationMode) – Desired interpolation enum defined by torchvision.transforms.InterpolationMode. Default is InterpolationMode.BILINEAR. If input is Tensor, only InterpolationMode.NEAREST, InterpolationMode.NEAREST_EXACT, InterpolationMode.BILINEAR and InterpolationMode.BICUBIC are supported. The corresponding Pillow integer constants, e.g. PIL.Image.BILINEAR are accepted as well.

        max_size (int, optional) – The maximum allowed for the longer edge of the resized image. If the longer edge of the image is greater than max_size after being resized according to size, size will be overruled so that the longer edge is equal to max_size. As a result, the smaller edge may be shorter than size. This is only supported if size is an int (or a sequence of length 1 in torchscript mode).

        antialias (bool, optional) –

        Whether to apply antialiasing. It only affects tensors with bilinear or bicubic modes and it is ignored otherwise: on PIL images, antialiasing is always applied on bilinear or bicubic modes; on other modes (for PIL images and tensors), antialiasing makes no sense and this parameter is ignored. Possible values are:

            True (default): will apply antialiasing for bilinear or bicubic modes. Other mode aren’t affected. This is probably what you want to use.

            False: will not apply antialiasing for tensors on any mode. PIL images are still antialiased on bilinear or bicubic modes, because PIL doesn’t support no antialias.

            None: equivalent to False for tensors and True for PIL images. This value exists for legacy reasons and you probably don’t want to use it unless you really know what you are doing.

        The default value changed from None to True in v0.17, for the PIL and Tensor backends to be consistent.

 forward(img)


    Parameters:

        img (PIL Image or Tensor) – Image to be scaled.
    Returns:

        Rescaled image.
    Return type:

        PIL Image or Tensor
```

**解释如下：**
在PyTorch官方文档中，有两个不同的“Parameters”部分，分别用于解释torchvision.transforms.Resize方法的参数和forward方法的参数。这两个方法是在使用图像转换时的不同步骤和目的。
+ torchvision.transforms.Resize的Parameters:
 >size (sequence or int)：指定调整后的目标尺寸。如果size是一个序列（如(h, w)），则输出图像将与此尺寸匹配。如果size是一个整数，则图像的较短边将与此数字匹配，并相应地调整另一边，保持图像的比例不变。   
interpolation (InterpolationMode)：指定用于调整大小时的插值方式。默认是双线性插值（BILINEAR）。只有特定的插值模式适用于张量，而Pillow（PIL）支持更多类型的插值。   
max_size (int, optional)：设置调整大小后图像较长边的最大允许长度。如果按照size设置后图像的较长边超过max_size，则会重设尺寸以使较长边等于max_size。   
antialias (bool, optional)：是否应用抗锯齿。这只影响采用双线性或双三次插值模式的张量，对PIL图像无需设置，因为PIL在这些模式下默认进行抗锯齿处理   

+ forward方法的Parameters: img (PIL Image or Tensor)：需要被调整大小的图像。此参数可以是PIL图像或张量格式。   
>forward方法是Resize类的实际调用方法，用于执行图像缩放操作。你在使用Resize转换实例对图像进行操作时，通过调用这个方法来将之前设置的参数应用于具体的图像，实现调整大小的功能。

简单来说，Resize的参数用于定义如何改变图像的大小（包括尺寸、插值方法等），而forward方法则是用这些定义来具体操作输入的图像。


In [2]:
image_path = "./dog.jpeg"                     # 需要先对原先的图片作resize操作，CIFAR10接受的模型输入是32*32
image = Image.open(image_path)      

image = image.convert('RGB')
transfrom = torchvision.transforms.Compose([ torchvision.transforms.Resize((32, 32)),
                                                                                                 torchvision.transforms.ToTensor()])

image = transfrom(image)
print(image.shape)

image = torch.reshape(image, (1, 3, 32, 32))
print(image.shape)

torch.Size([3, 32, 32])
torch.Size([1, 3, 32, 32])


### CPU训练出的模型导入

In [3]:
# 这个方法必须是CPU条件下训练出来的模型：
model = torch.load("../L27_L29/CAFIR10_ckpt_e10.pth")
print(model)

Tudui(
  (model): Sequential(
    (0): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Flatten()
    (7): Linear(in_features=1024, out_features=64, bias=True)
    (8): Linear(in_features=64, out_features=10, bias=True)
  )
)


In [4]:
model.eval()
with torch.no_grad():
    output = model(image)
    print(output)

tensor([[-2.9093, -2.8773,  1.8954,  2.2320,  0.2233,  3.7644, -0.9736,  1.4850,
         -1.8810, -1.9220]])


In [5]:
print(output.argmax(1))

tensor([5])


☝ 第五个标签恰好是🐶，说明模型预测正确

### GPU训练出的模型导入
+ **不同pytorch版本下训练出来的模型不能用其他版本的pytorch进行导入推理**

In [10]:
model = torch.load("../L27_L29/CAFIR10_ckpt_e10.pth", map_location=torch.device('cpu'))              # 只需要在map_location修改映射的device就ok
print(model)

Tudui(
  (model): Sequential(
    (0): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Flatten()
    (7): Linear(in_features=1024, out_features=64, bias=True)
    (8): Linear(in_features=64, out_features=10, bias=True)
  )
)


In [11]:
model.eval()
with torch.no_grad():
    output = model(image)
    print(output)

tensor([[-2.9093, -2.8773,  1.8954,  2.2320,  0.2233,  3.7644, -0.9736,  1.4850,
         -1.8810, -1.9220]])


In [12]:
print(output.argmax(1))

tensor([5])
